In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

C:\Users\harkum\AppData\Local\Temp\ipykernel_32256\1771248598.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
new_df = df.sample(30000)

In [4]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
new_df.duplicated().sum() # There are No Duplications 

0

In [7]:
ques_df = new_df[['question1','question2']]

In [9]:
ques_df.head()

,question1,question2
15701,What are the best romance movies?,Which are the most romantic movies?
245713,What is it like to live in South Africa?,What's it like to live in South Africa?
41178,What are the meanings of Punjabi words 'Varga'...,"What is the meaning of the Punjabi word ""pendu""?"
82652,How much can a wine shop owner earn in a year ...,How much can a cafe shop earn in an year?
35081,Do many Mexican women like East Asian men (Kor...,Are there many Mexican women that like East As...


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# Create the Text Corpus 
questions = list(ques_df['question1'])+list(ques_df['question2']) 
cv = CountVectorizer(max_features=3000)
q1_arr,q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [11]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [12]:
temp_df 

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
15701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
82652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
165738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202524,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
392430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [15]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7508333333333334

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)